## Create a script to update the JSON file for the D3 unrealized gains/losses chart

Import all the necessary modules and utilities. Make sure that you have these modules in your python environment. 

In [ ]:
from datetime import datetime as dt
from datetime import timedelta as tdelta
from yahoo_finance import Share
import pandas as pd
from dateutil.parser import parse as dtparse
import json
from pprint import pprint
import numpy as np

We need to create a couple of functions that will create the offset for each box segment start location and sort the values of each day's gains/losses.

1. The offset_list function creates an additional column that computes the start points for the box offset in each day's gains/losses. This will be used to properly stack those gains and losses in the D3 chart.

In [ ]:
def offset_list(ldf):
    offset = 0
    offsetlist = []
    fneg=True
    for items in ldf:
        if fneg:
            if items>0:
                offsetlist.append(offset)
                offset=np.abs(offset)+np.abs(items)
            else:
                offset=0
                offsetlist.append(offset)
                offset=np.abs(offset)+np.abs(items)
                fneg=False
        else:
            offsetlist.append(offset)
            offset=np.abs(offset)+np.abs(items)    
        return offsetlist


2. We first need to send the sorted values of each date entry's gains/losses. In order for the offset to make it so that the maximum values are below the others (in the positive and negative sections), we arrange them so that the values are sorted from maximum to minimum. 

In [ ]:
def sorted_values(pddf,i):
    df =  pddf.ix[i].sort_values(ascending=False)
    ofsl = offset_list(df)
    dfret = pd.DataFrame(columns=['name','value','offset'])
    dfret['name'] = df.index
    dfret['value'] = list(df)
    dfret['offset'] = ofsl
    dfret['date'] = df.name
    return dfret


Within a given range, we want to know if the market was open on the previous days and back 15 days. We chose one of the oldest stocks and check the return of the historical information for that range. This will return records for data of when the market was open. Instead of only excluding weekend, this will also include bank holidays, for example. 

In [ ]:
yesterday = dt(dt.now().year, dt.now().month, dt.now().day, 12,0,0) - tdelta(1)
fiftdback = yesterday - tdelta(15)

hist_ED = Share("ED").get_historical(fiftdback.isoformat().split("T")[0],yesterday.isoformat().split("T")[0])

newdatelist = []
for k in range(len(hist_ED)):
        newdatelist.append(hist_ED[len(hist_ED)-(k+1)]['Date'])

Need to compare details about the dates.
We want to have populated a JSON file that contains the stock information
So we need to compare first the size of the JSON list.
1. Assess the following cases:
      a. On a new day, does the end-date of the newdatelist increase the range of dates? If so, we need to add new data and subtract from the JSON file.
      b. On a new day, does the end-date of the new datelist do nothing to the range of dates? Then we need to do nothing.
2. We need to access the dates in the JSON file in order to make this work.

In [ ]:
with open('stocks.json') as data_file:    
    data = json.load(data_file)

current_datelist = []
for k in range(len(data)):
    current_datelist.append(data[k][str(k)][0]['date'])

# 3. Compare the end-dates

daydiff = dtparse(newdatelist[-1]) - dtparse(current_datelist[-1])

# 4. If the daydiff >=1, then there needs to subtract a day, and advance the day.
if (daydiff.days >= 1):
    data2 = []
    # This will subtract the last entry in the JSON file and move the entries down to
    # to make space for the newest entry
    for k in range(len(data)-1): 
        dt2 = {}
        dt2[str(k)] = data[k+1][str(k+1)]
        data2.append(dt2)

    # This will read the list of owned stocks and their value and share prices
    stocklist = pd.read_csv("stock_list_today.csv")
    stocklist['value'] = stocklist['shareprice']*stocklist['shares']
    dfst = stocklist.set_index('name')
    df = pd.DataFrame(columns = ['date'])
    df['date'] = [newdatelist[-1]]

    for stocks in stocklist['name']:
        df[stocks] = df['date'].apply(lambda x: np.float(Share(stocks).get_historical(x,x)[0]['Close'])*dfst.transpose()[stocks]['shares'] - dfst.transpose()[stocks]['value'])
    df = df.set_index('date')
    df = df.sort_index()
    df0 = sorted_values(df,0)
    df1 =  json.loads(df0.to_json(orient='records'))

    for item in df1:
        item.pop('date')
    df2 = {} ; df2['date'] = json.loads(df0.to_json(orient='records'))[0]['date']
    df1.append(df2)
    df1.reverse()

    df2 = {} ; df2[str(k+1)] = df1
    data2.append(df2)
    f = open('stocks.json','w')
    f.write(json.dumps(data2)) 
    f.close()